### Let's try data Augmentation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.io

In [2]:
train = scipy.io.loadmat('train_32x32.mat')
test  = scipy.io.loadmat('test_32x32.mat')

In [3]:
train['X'] = train['X'][np.newaxis,...]
train_data = np.swapaxes(train['X'],0,4).squeeze()

test['X'] = test['X'][np.newaxis,...]
test_data = np.swapaxes(test['X'],0,4).squeeze()

In [4]:
train_data = train_data.astype('float32')
test_data = test_data.astype('float32')

train_data /= 255.0
test_data /= 255.0

##### convert rgb to grayscale

In [5]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

X_train = np.zeros((73257, 32, 32))
for i in range(train_data.shape[0]):
    X_train[i] = rgb2gray(train_data[i])
    
X_test = np.zeros((26032, 32, 32))
for i in range(test_data.shape[0]):
    X_test[i] = rgb2gray(test_data[i])

In [6]:
X_train = X_train.reshape(X_train.shape[0],32, 32, 1)
X_test = X_test.reshape(X_test.shape[0], 32, 32, 1)

----------------------------------

In [18]:
import keras
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

---------------------------

In [19]:
y_train = keras.utils.to_categorical(train['y'][:,0])[:,1:]
y_test = keras.utils.to_categorical(test['y'][:,0])[:,1:]

--------------------------

In [24]:
print("Augmenting training set images...")
datagen = ImageDataGenerator(
width_shift_range=0.2,
height_shift_range=0.15,
shear_range=0.4,
zoom_range=[1.0,1.6],)

Augmenting training set images...


-------------------------------

In [25]:
model5 = Sequential()
model5.add(Conv2D(32, (3,3), strides=(1,1), padding='same', input_shape=(32,32,1)))
model5.add(Activation('relu'))
model5.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model5.add(Dropout(0.35))
model5.add(Conv2D(64, (3,3), strides=(1,1),padding='same'))
model5.add(Activation('relu'))
model5.add(Conv2D(64, (3,3), strides=(1,1),padding='same'))
model5.add(Activation('relu'))
model5.add(MaxPooling2D(pool_size=(2,2), padding='same', strides=(2,2)))
model5.add(Dropout(0.35))
model5.add(Flatten())
model5.add(Dense(256))
model5.add(Activation('relu'))
model5.add(Dropout(0.35))
model5.add(Dense(10))
model5.add(Activation('softmax'))
model5.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 32, 32, 32)        320       
_________________________________________________________________
activation_21 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
activation_22 (Activation)   (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 16, 16, 64)        36928     
__________

In [26]:
adam = Adam(decay=1e-6)
model5.compile(loss='categorical_crossentropy', optimizer=adam , metrics=['accuracy'])

In [27]:
history = model.fit_generator(datagen.flow(X_train, y_train, batch_size=128), steps_per_epoch= (X_train.shape[0]/128), epochs= 15, verbose= 1)

Epoch 1/15
573/572 [==============================] - 418s - loss: 1.8468 - acc: 0.3546   
Epoch 2/15
573/572 [==============================] - 415s - loss: 1.2580 - acc: 0.5822   
Epoch 3/15
573/572 [==============================] - 411s - loss: 1.0630 - acc: 0.6544   
Epoch 4/15
573/572 [==============================] - 411s - loss: 0.9587 - acc: 0.6900   
Epoch 5/15
573/572 [==============================] - 415s - loss: 0.8920 - acc: 0.7138   
Epoch 6/15
573/572 [==============================] - 417s - loss: 0.8341 - acc: 0.7307   
Epoch 7/15
573/572 [==============================] - 414s - loss: 0.7884 - acc: 0.7472   
Epoch 8/15
573/572 [==============================] - 413s - loss: 0.7557 - acc: 0.7590   
Epoch 9/15
573/572 [==============================] - 412s - loss: 0.7224 - acc: 0.7725   
Epoch 10/15
573/572 [==============================] - 413s - loss: 0.6985 - acc: 0.7788   
Epoch 11/15
573/572 [==============================] - 412s - loss: 0.6798 - acc: 0.7853 

In [28]:
score5 = model5.evaluate(X_test, y_test, verbose=0)
print('loss:', score5[0])
print('Test accuracy:', score5[1])

loss: 0.288844556523
Test accuracy: 0.923632452366
